# Exploration of some plotting utils

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import plotters
from PIL import Image, ImageFilter

im_pil = Image.open('docs/trinity.jpg')
im = np.array(im_pil)

# Scaling and displaying float images
Matplotlib's `imshow` works well for colour images which are 8-bit unsigned integers (i.e. in the range 0 to 255).

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5), num=1)
ax.imshow(im)
plt.show()

It can also handle float arrays if they are monochrome, regardless of what scale the values have

In [ ]:
bw = im[:,:,1].astype('float')
bw2 = bw*0.1 - 50

fig, ax = plt.subplots(1, 2, figsize=(10,5), num=2)
ax[0].imshow(bw, cmap='gray')
ax[1].imshow(bw2, cmap='gray')
plt.show()

However floating 3 channel images have to be in the range 0 to 1. If not, you'll get a weird looking image

In [ ]:
f_im = im.astype('float')
f_im2 = f_im / 255.0

fig, ax = plt.subplots(1, 2, figsize=(10,5), num=3)
ax[0].imshow(f_im, cmap='gray')
ax[1].imshow(f_im2, cmap='gray')
plt.show()

This leads us to the first function in the plotters module - `imshowNormalize`. This will take a colour input of any scale and scale it to be in the range 0 to 1

In [ ]:
f_im = im.astype('float')
f_im = f_im * 0.2 + 32

fig, ax = plt.subplots(1, figsize=(10,5), num=4)
ax.imshow(plotters.imshowNormalize(f_im))
plt.show()

Instead of calling this and then calling `imshow`, you can also call `plotters.imshow`, which will call it automatically for you. It also removes the tick labels and scales the image to fit the figure.

In [ ]:
f_im = im.astype('float')
f_im = f_im * 0.2 + 32

fig, ax = plt.subplots(1, figsize=(6,4), num=5)
plotters.imshow(f_im, ax)
plt.show()

# Side by Side Plotting
Let us blur an image to imitate a filtering operation we want to examine more closely.

In [ ]:
im1 = np.array(im_pil.filter(ImageFilter.BLUR))

fig, ax = plt.subplots(1, figsize=(6,4), num=6)
plotters.imshow(im1, ax)
plt.show()

Plotting side by side isn't hard - we did it above with a call to `plt.subplots(1,2, ...)`. Again it does get annoying though having to worry about scaling if both images are floats; as well as removing the ticks and positioning them nicely. The `plotters.plot_sidebyside` function will handle this automatically.

In [ ]:
f_im1 = im1.astype('float') * 0.5 - 60
f_im = im.astype('float') * 0.3 + 100

fig, axes = plt.subplots(1, 2, figsize=(8,4), num=7)
plotters.plot_sidebyside(f_im, f_im1, axes)
plt.show()

You can also zoom into a particular part of the image and display this side by side, again with all the extra convenience of handling floats nicely.

In [ ]:
# Let us zoom in on the fountain. This is roughly at pixel location (325, 1080)
centre = (325, 1080)
fig, axes = plt.subplots(1, 2, figsize=(8,4), num=8)
plotters.zoom_sidebyside(f_im, f_im1, centre=centre, size=200, axes=axes)

You can also feed it centre positions in the range 0 to 1. The zoom_sidebyside function determines how to interpret the centre positions by whether they are ints (assumes pixel coords) or floats (assumes relative position)

In [ ]:
centre = (0.339, 0.844)
fig, axes = plt.subplots(1, 2, figsize=(8,4), num=9)
plotters.zoom_sidebyside(f_im, f_im1, centre=centre, size=200, axes=axes)

# Displaying grids of data

## A 3d 'tensor' of images
This often happens in CNNs. You may want to examine an activation layer which may be $100 \times 100 \times 16$ say. We can view them as a grid of black and white images with the `plot_activations` function

In [ ]:
# Simulate an array of activations - this next line grabs 16 patches from the 
# above image by randomly sampling a row position, col position and colour channel.
rows = np.random.randint(300, im1.shape[0]-50, size=(16,)) # We sample from 300 to avoid all the sky in the image
cols = np.random.randint(50, im1.shape[1]-50, size=(16,))
channels = np.random.randint(0, 3, size=(16,))
acts = [im[r-50:r+50,c-50:c+50,ch] for r,c,ch in zip(rows, cols, channels)]

# Convert the list to a numpy array by stacking the elements
acts = np.stack(acts, axis=-1)
acts.shape

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), num=10)
plotters.plot_activations(acts, cols=4, draw=True, ax=ax);

By default, all of the images will be scaled individually, but you may not want to do this. You can change this behaviour by scaling the image as a whole.

In [ ]:
acts = acts * np.linspace(0.2, 0.8, 16)

fig, ax = plt.subplots(1, figsize=(4,4), num=11)
plotters.plot_activations(acts, cols=4, draw=True, scale_individual=False, ax=ax);

## A 4d 'tensor' (a batch of colour images)
It often happens that I will want to look at a batch of RGB images (e.g reconstructions from activations in a CNN). I.e. we will have an array of shape [N, H, W, 3], and I'll want to view all of these as a grid. To do this, we can use `plotters.plot_batch_colour`.

In [ ]:
# Simulate an array of colour patches - this next line grabs 16 patches from the 
# above image by randomly sampling a row position and col position
rows = np.random.randint(300, im1.shape[0]-50, size=(16,)) # We sample from 300 to avoid all the sky in the image
cols = np.random.randint(50, im1.shape[1]-50, size=(16,))
acts = [im[r-50:r+50,c-50:c+50,:] for r,c in zip(rows, cols)]

# Convert the list to a numpy array by stacking the elements
batch = np.stack(acts, axis=0)
batch.shape

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), num=12)
plotters.plot_batch_colour(batch, cols=4, draw=True, ax=ax);

Again, we can scale the images individually (as above) or as a whole (below)

In [ ]:
batch2 = batch * np.reshape(np.linspace(0.8, 0.2, 16), [16, 1, 1, 1])

fig, ax = plt.subplots(1, figsize=(4,4), num=13)
plotters.plot_batch_colour(batch2, cols=4, draw=True, scale_individual=False, ax=ax);

## Zooming in on a 4d array of images
Say we wanted to look at the centre point of all of these 16 images. We can do use `plotters.zoom_batch_colour`. To plot them, we can then use the above plotting methods.

In [ ]:
centre = (0.4, 0.5) # can also feed in an array of size (16,2) to specify centre locations for each image
batch2 = plotters.zoom_batch_colour(batch, (0.4, 0.5), size=20)

fig, ax = plt.subplots(1, figsize=(4,4), num=14)
plotters.plot_batch_colour(batch2, cols=4, draw=True, scale_individual=False, ax=ax);

## Creating a grid of axes to plot individually to
An alternative to the above method of plotting multiple images (internally, this creates one large big image and plots it on a single axis) is to create mutliple axes and plot to them manually. Matplotlib provides the gridspec package to help you do this. I do this often so I've created the function `plot_axgrid` to give me axis handles. You can feed it all the standard figure keyword args like figsize and num too.

In [ ]:
fig, axes= plotters.plot_axgrid(4,4, figsize=(6,6), num=15)
for i in range(4):
    for j in range(4):
        axes[i][j].imshow(batch[i*4+j])